<a href="https://colab.research.google.com/github/Arcee-LYK/mnist_tutorial/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [4]:
import torch.nn.functional as F
import torch.optim as optim
class SimpleNet(nn.Module):
  def __init__(self):
    super(SimpleNet,self).__init__()
    self.conv1 = nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = nn.Conv2d(10,20,kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320,50)
    self.fc2 = nn.Linear(50,10)
  def forward(self,x):
    x = F.relu(F.max_pool2d(self.conv1(x),2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
    x = x.view(-1,320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x,training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(),lr=0.01,momentum=0.5)

Next, we can start to train and evaluate!

In [7]:
from torch.autograd import Variable
# train and evaluate
train_accuracy = []
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images,labels = Variable(images).cuda(),Variable(labels).cuda()
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        # evaluate
        # TODO:calculate the accuracy using traning and testing dataset
        correct = 0
        size = 0
        images,labels = Variable(images).cuda(),Variable(labels).cuda()
        outputs = net(images)
        predict = outputs.data.max(1)[1]
        size += len(predict)
        correct += predict.eq(labels.data).sum()
    train_accuracy.append(float(100*correct/size))
for epoch in range(NUM_EPOCHS):
  print("epoch",epoch,"train accuracy:%0.2f" % (train_accuracy[epoch]))

net.eval()
correct = 0
for data_test in test_loader:
  images ,labels = data_test
  images,labels = Variable(images).cuda(),Variable(labels).cuda()
  output_test = net(images)
  _,predict = torch.max(output_test,1)
  correct += (predict == labels).sum()
print("Test acc: {0}".format(correct.item() / len(test_dataset)))

100%|██████████| 468/468 [00:08<00:00, 55.50it/s]


epoch 0 train accuracy:96.09
epoch 1 train accuracy:98.44
epoch 2 train accuracy:99.22
epoch 3 train accuracy:94.53
epoch 4 train accuracy:100.00
epoch 5 train accuracy:99.22
epoch 6 train accuracy:98.44
epoch 7 train accuracy:99.22
epoch 8 train accuracy:99.22
epoch 9 train accuracy:99.22
Test acc: 0.9854


#### Q5:
Please print the training and testing accuracy.